In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [3]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'DummyClassifier_gl_users_Timeline_test_results.csv',
 'DummyClassifier_ig_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_ig_users_Stance_test_results.csv',
 'DummyClassifier_ig_users_Timeline_test_results.csv',
 'DummyClassifier_lu_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_lu_users_Stance_test_r

In [4]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hidroxicloroquina',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hidroxicloroquina', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

In [5]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"DummyClassifier", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base-uncased" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base-uncased", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"DummyClassifier", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base-uncased" ,"xgboost", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"DummyClassifier", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base-uncased" ,"xgboost", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv")
    
]

list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        print(text_col, vectorizer, estimator,target)
        print(path)
        display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

Stance - DummyClassifier ig
../reports/test_results/DummyClassifier_ig_users_Stance_test_results.csv


metric                                           precision    recall  \
target                                              Church    Church   
text_col vectorizer estimator       class                              
Stance   -          DummyClassifier against       0.565943  1.000000   
                                    accuracy      0.565943  0.565943   
                                    weighted avg  0.320292  0.565943   
                                    macro avg     0.282972  0.500000   
                                    for           0.000000  0.000000   

metric                                            f1-score     support  
target                                              Church      Church  
text_col vectorizer estimator       class                               
Stance   -          DummyClassifier against       0.722814  339.000000  
                                    accuracy      0.565943    0.565943  
                                    weighted avg  0.409072  599.000000  
                                    macro avg     0.361407  599.000000  
                                    for           0.000000  260.000000

Stance - DummyClassifier bo
../reports/test_results/DummyClassifier_bo_users_Stance_test_results.csv


metric                                           precision    recall  \
target                                           Bolsonaro Bolsonaro   
text_col vectorizer estimator       class                              
Stance   -          DummyClassifier against       0.861702  1.000000   
                                    accuracy      0.861702  0.861702   
                                    weighted avg  0.742531  0.861702   
                                    macro avg     0.430851  0.500000   
                                    for           0.000000  0.000000   

metric                                            f1-score     support  
target                                           Bolsonaro   Bolsonaro  
text_col vectorizer estimator       class                               
Stance   -          DummyClassifier against       0.925714  162.000000  
                                    accuracy      0.861702    0.861702  
                                    weighted avg  0.797690  188.000000  
                                    macro avg     0.462857  188.000000  
                                    for           0.000000   26.000000

Stance - DummyClassifier cl
../reports/test_results/DummyClassifier_cl_users_Stance_test_results.csv


...

metric                                                   precision    recall  \
target                                                     Sinovac   Sinovac   
text_col vectorizer               estimator class                              
Timeline bertabaporu-base-uncased xgboost   for           0.771552  0.852381   
                                            accuracy      0.782946  0.782946   
                                            weighted avg  0.784563  0.782946   
                                            macro avg     0.785776  0.776473   
                                            against       0.800000  0.700565   

metric                                                    f1-score     support  
target                                                     Sinovac     Sinovac  
text_col vectorizer               estimator class                               
Timeline bertabaporu-base-uncased xgboost   for           0.809955  420.000000  
                                            accuracy      0.782946    0.782946  
                                            weighted avg  0.781156  774.000000  
                                            macro avg     0.778471  774.000000  
                                            against       0.746988  354.000000

Timeline bertabaporu-base-uncased xgboost gl
../reports/test_results/bertimbau_xgb_gl_users_emb_Timeline_test_results.csv


metric                                                   precision    recall  \
target                                                    Globo TV  Globo TV   
text_col vectorizer               estimator class                              
Timeline bertabaporu-base-uncased xgboost   for           0.652921  0.778689   
                                            accuracy      0.622871  0.622871   
                                            weighted avg  0.611101  0.622871   
                                            macro avg     0.601460  0.586949   
                                            against       0.550000  0.395210   

metric                                                    f1-score     support  
target                                                    Globo TV    Globo TV  
text_col vectorizer               estimator class                               
Timeline bertabaporu-base-uncased xgboost   for           0.710280  244.000000  
                                            accuracy      0.622871    0.622871  
                                            weighted avg  0.608557  411.000000  
                                            macro avg     0.585105  411.000000  
                                            against       0.459930  167.000000

Timeline bertabaporu-base-uncased xgboost lu
../reports/test_results/bertimbau_xgb_lu_users_emb_Timeline_test_results.csv


metric                                                   precision    recall  \
target                                                        Lula      Lula   
text_col vectorizer               estimator class                              
Timeline bertabaporu-base-uncased xgboost   against       0.648148  0.734266   
                                            accuracy      0.650735  0.650735   
                                            weighted avg  0.651182  0.650735   
                                            macro avg     0.651347  0.646203   
                                            for           0.654545  0.558140   

metric                                                    f1-score     support  
target                                                        Lula        Lula  
text_col vectorizer               estimator class                               
Timeline bertabaporu-base-uncased xgboost   against       0.688525  143.000000  
                                            accuracy      0.650735    0.650735  
                                            weighted avg  0.647731  272.000000  
                                            macro avg     0.645518  272.000000  
                                            for           0.602510  129.000000

In [6]:
df_results_final

metric                                                                  precision  \
target                                                                     Church   
text_col vectorizer               estimator                class                    
Stance   -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
                                                           for           0.000000   
         TF-IDF                   xgboost                  against       0.732394   
                                                           accuracy      0.709516   
                                                           weighted avg  0.708016   
                                                           macro avg     0.704312   
                                                           for           0.676230   
         bertabaporu-base-uncased xgboost                  against       0.732394   
                                                           accuracy      0.709516   
                                                           weighted avg  0.708016   
                                                           macro avg     0.704312   
                                                           for           0.676230   
         -                        bertabaporu-base-uncased against       0.850267   
                                                           accuracy      0.871452   
                                                           weighted avg  0.874748   
                                                           macro avg     0.878467   
                                                           for           0.906667   
Texts    -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
                                                           for           0.000000   
         TF-IDF                   xgboost                  against       0.638677   
                                                           accuracy      0.616027   
                                                           weighted avg  0.610089   
                                                           macro avg     0.605746   
                                                           for           0.572816   
         bertabaporu-base-uncased xgboost                  against       0.639474   
                                                           accuracy      0.611018   
                                                           weighted avg  0.605691   
                                                           macro avg     0.600559   
                                                           for           0.561644   
Timeline -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
                                                           for           0.000000   
         TF-IDF                   xgboost                  against       0.726519   
                                                           accuracy      0.707846   
                                                           weighted avg  0.706034   
                                                           macro avg     0.702922   
     

In [7]:

df_results_final.to_excel("~/teste.xlsx")

In [8]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                                f1-score  \
target                                                                  Church   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.702021   
         bertabaporu-base-uncased xgboost                  macro avg  0.702021   
         -                        bertabaporu-base-uncased macro avg  0.866621   
Texts    -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.596115   
         bertabaporu-base-uncased xgboost                  macro avg  0.594754   
Timeline -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.699122   
         bertabaporu-base-uncased xgboost                  macro avg  0.661773   

metric                                                                          \
target                                                               Bolsonaro   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.595312   
         bertabaporu-base-uncased xgboost                  macro avg  0.595312   
         -                        bertabaporu-base-uncased macro avg  0.739125   
Texts    -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.501220   
         bertabaporu-base-uncased xgboost                  macro avg  0.495578   
Timeline -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.728220   
         bertabaporu-base-uncased xgboost                  macro avg  0.692262   

metric                                                                                  \
target                                                               Hidroxicloroquina   
text_col vectorizer               estimator                class                         
Stance   -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.734366   
         bertabaporu-base-uncased xgboost                  macro avg          0.734366   
         -                        bertabaporu-base-uncased macro avg          0.844778   
Texts    -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.603727   
         bertabaporu-base-uncased xgboost                  macro avg          0.609524   
Timeline -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.898944   
         bertabaporu-base-uncased xgboost                  macro avg          0.776979   

metric                                                                          \
target                                                                 Sinovac   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.351759   
         TF-IDF                   xgboost                  macro avg  0.752640   
         bertabaporu-base-uncased xgboost                  macro avg  0.752640   
         -                        bertabaporu-base-uncased macro avg  0.840499   
Texts    -                        DummyClassifier          macro avg  0.351759   
         TF-IDF                   xgboo

In [9]:
f1_macro_df.to_excel("~/teste.xlsx")

In [10]:
f1_macro_df.sort_values(('f1-score','overall'))

metric                                                                f1-score  \
target                                                                  Church   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.361407   
Texts    -                        DummyClassifier          macro avg  0.361407   
Timeline -                        DummyClassifier          macro avg  0.361407   
Texts    bertabaporu-base-uncased xgboost                  macro avg  0.594754   
         TF-IDF                   xgboost                  macro avg  0.596115   
Stance   TF-IDF                   xgboost                  macro avg  0.702021   
         bertabaporu-base-uncased xgboost                  macro avg  0.702021   
Timeline bertabaporu-base-uncased xgboost                  macro avg  0.661773   
         TF-IDF                   xgboost                  macro avg  0.699122   
Stance   -                        bertabaporu-base-uncased macro avg  0.866621   

metric                                                                          \
target                                                               Bolsonaro   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.462857   
Texts    -                        DummyClassifier          macro avg  0.462857   
Timeline -                        DummyClassifier          macro avg  0.462857   
Texts    bertabaporu-base-uncased xgboost                  macro avg  0.495578   
         TF-IDF                   xgboost                  macro avg  0.501220   
Stance   TF-IDF                   xgboost                  macro avg  0.595312   
         bertabaporu-base-uncased xgboost                  macro avg  0.595312   
Timeline bertabaporu-base-uncased xgboost                  macro avg  0.692262   
         TF-IDF                   xgboost                  macro avg  0.728220   
Stance   -                        bertabaporu-base-uncased macro avg  0.739125   

metric                                                                                  \
target                                                               Hidroxicloroquina   
text_col vectorizer               estimator                class                         
Stance   -                        DummyClassifier          macro avg          0.334878   
Texts    -                        DummyClassifier          macro avg          0.334878   
Timeline -                        DummyClassifier          macro avg          0.334878   
Texts    bertabaporu-base-uncased xgboost                  macro avg          0.609524   
         TF-IDF                   xgboost                  macro avg          0.603727   
Stance   TF-IDF                   xgboost                  macro avg          0.734366   
         bertabaporu-base-uncased xgboost                  macro avg          0.734366   
Timeline bertabaporu-base-uncased xgboost                  macro avg          0.776979   
         TF-IDF                   xgboost                  macro avg          0.898944   
Stance   -                        bertabaporu-base-uncased macro avg          0.844778   

metric                                                                          \
target                                                                 Sinovac   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.351759   
Texts    -                        DummyClassifier          macro avg  0.351759   
Timeline -                        DummyClassifier          macro avg  0.351759   
Texts    bertabaporu-base-uncased xgboost                  macro avg  0.660542   
         TF-IDF                   xgboost                  macro avg  0.670805   
Stance   TF-IDF                   xgboo